<a href="https://colab.research.google.com/github/kguzmand/MetNumUN2023II/blob/main/Week4IterativeMethodsForLinearSystemsGroup3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GROUP 3**

Name
*  Nicolas Felipe Arciniegas Lizarazo
*  Karen Lorena Guzman del Rio
*  Sebastian Lopez Silva

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [3]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [4]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [5]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [6]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [7]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [8]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [23]:
import numpy as np

def jIteration(MA, Vb, num_iteration):
  n = MA.shape[0]
  x = np.zeros(n)
  norm_dif = []
  norm_B = []
  for _ in range(num_iteration):
      x_new = np.zeros(n)
      for i in range(n):
          x_new[i] = (Vb[i] - np.dot(MA[i, :i], x[:i]) - np.dot(MA[i, i+1:], x[i+1:])) / MA[i, i]
      norm_dif.append(np.linalg.norm(x_new - x))
      norm_B.append(np.linalg.norm(MA - np.diag(np.diagonal(MA))))

      x = x_new

  return x, norm_dif, norm_B

In [18]:
A = np.array([[6, 1, 2],
              [1, 7, 4],
              [1, 6, 8]])

b = np.array([1, 3, 5])

numIteration = 10

In [20]:
x, dif, normB = jIteration(A, b, numIteration)

In [21]:
A_modif = np.array([[0.6, 0.1, 0.2],
                       [0.1, 0.7, 0.4],
                       [0.1, 0.6, 0.8]])
x_modif, dif_modif, norm_B_modif = jIteration(A_modif, b, numIteration)

In [22]:
print('Matriz original')
print(x)
print(dif)

print(normB)

print('Matriz modificada')
print(x_modif)
print(dif_modif)

print(norm_B_modif)

Matriz original
[-0.04188282  0.11678153  0.51352992]
[0.7759357236044316, 0.5835534691530367, 0.4357224726804294, 0.32314853072724203, 0.24256194002189843, 0.18014613587801545, 0.13508476487505122, 0.10047213491478979, 0.07527220440719731, 0.056045507683387855]
[7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608, 7.681145747868608]
Matriz modificada
[-0.41882819  1.16781534  5.1352992 ]
[7.759357236044316, 5.835534691530367, 4.3572247268042945, 3.231485307272421, 2.4256194002189853, 1.8014613587801551, 1.3508476487505126, 1.0047213491478983, 0.752722044071974, 0.5604550768338799]
[0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608, 0.7681145747868608]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [26]:
import numpy as np

def gsIteration(A, b, numIteration):
    n = A.shape[0]
    x = np.zeros(n)
    normDif = []
    normB = []
    for _ in range(numIteration):
        X = np.zeros(n)
        for i in range(n):
            X[i] = (b[i] - np.dot(A[i, :i], X[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        normDif.append(np.linalg.norm(X - x))
        normB.append(np.linalg.norm(A - np.tril(A) @ np.triu(A, 1)))

        x = X

    return x, normDif, normB

In [30]:
n = 6
A = np.random.rand(n, n)
b = np.random.rand(n)
numIteration = 10

In [31]:
x, dif, normB = gsIteration(A, b, numIteration)
print(x)
print(dif)
print(normB)

[  29.19326527 -111.58020836  187.42584039 -149.01707753   59.77460197
  -83.11558723]
[1.509654687847105, 2.4918910555563594, 2.295488355534375, 8.768807826778366, 18.64910084232208, 17.83065784991639, 57.72194888017752, 143.24546452821116, 153.83135775611382, 368.26477654003486]
[2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685, 2.9377446351949685]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [37]:
def minres (A, b, num_iterations):
    n = A.shape[0]
    x = np.zeros(n)
    r = b - A @ x
    p = r.copy()
    normR = []
    normD = []
    tauValues = []

    for k in range(numIteration):
        Ap = A @ p
        dem = np.dot(p, Ap)
        if dem == 0:
            break
        tau = np.dot(r, Ap) / dem
        X = x + tau * p
        R = r - tau * Ap
        normR.append(np.linalg.norm(R))
        normD.append(np.linalg.norm(X - x))
        tauValues.append(tau)
        x = X
        r = R
        beta = np.linalg.norm(r)
        p = r + (beta / np.linalg.norm(r)) * p

    return x, normR, normD, tauValues

In [38]:
n = 6
A = np.random.rand(n, n)
b = np.random.rand(n)
numIteration = 10

In [39]:
x_ground_truth = np.linalg.solve(A, b)
print(x_ground_truth)
xRes, normR, normD, tauValues = minres (A, b, numIteration)
print(xRes)
print(normR)
print(normD)
print(tauValues)

[ 0.79788027 -0.25913208 -0.3941368   0.47759792  0.26351923  0.22693771]
[-175.99014307  621.62344304   37.6138381  -597.6978035   -84.02369449
  278.0959235 ]
[2.703724213728299, 3.8112900249890944, 8.830298843558936, 13.070221480047366, 30.9699638618879, 47.673753554393365, 75.00064558450867, 206.02666216853052, 415.4894091956077, 778.0457606191125]
[1.475773914380681, 3.5324176209785647, 5.254517027184728, 10.741046659807985, 26.355110895674215, 35.99549103080048, 67.218229003138, 145.44618187767136, 306.28909094822575, 598.0295598488544]
[1.0, 1.6958464846193158, 1.6128296034239553, 1.283463664153764, 1.9722535862795765, 1.1034224830600659, 1.391330692134893, 1.5303554029108162, 1.15997031055533, 0.9497993800314544]
